In [ ]:
import pandas as pd
from edgar import *

set_identity("adunoorwork@gmail.com")

In [125]:
#Sales force cik - 0001108524
company = Company("0001108524")


In [165]:
q_filings = company.get_filings(form="10-Q")
k_filings = company.get_filings(form="10-K")


k_filings_1 = k_filings[0]
kf1 = k_filings_1.obj()

k_filings_2 = k_filings[1]
kf2 = k_filings_2.obj()

q_filings_1 = q_filings[0]
qf1 = q_filings_1.obj()

q_filings_2 = q_filings[1]
qf2 = q_filings_2.obj()

q_filings_3 = q_filings[2]
qf3 = q_filings_3.obj()

q_filings_4 = q_filings[3]
qf4 = q_filings_4.obj()

q_filings_5 = q_filings[4]
qf5 = q_filings_5.obj()

q_filings_6 = q_filings[5]
qf6 = q_filings_6.obj()

q_filings_7 = q_filings[6]
qf7 = q_filings_7.obj()

q_filings_8 = q_filings[7]
qf8 = q_filings_8.obj() 

In [167]:
import re
from datetime import datetime

def extract_financial_data(filing_text_current: str, filing_text_previous: str):
    """
    Extracts financial data for three fiscal years, using a second prior-year filing
    only for metrics that are not available for all three years in the current filing.

    Args:
        filing_text_current (str): The raw text of the most recent 10-K filing.
        filing_text_previous (str): The raw text of the 10-K filing from the prior year.

    Returns:
        dict: A dictionary with data for the three most recent fiscal years.
    """
    
    # Normalize both filing texts
    text_current = re.sub(r'\s+', ' ', filing_text_current)
    text_previous = re.sub(r'\s+', ' ', filing_text_previous)

    # --- Initialize lists for all variables (fixed syntax) ---
    dates, date_ids, periods = [None]*3, [None]*3, [None]*3
    total_revenues, cost_of_revenues, operating_incomes = [None]*3, [None]*3, [None]*3
    net_incomes, rd_expenses, sm_expenses, ga_expenses = [None]*3, [None]*3, [None]*3, [None]*3
    subscription_revenues, cost_of_subscription_revenues, prof_services_revenues, operating_cash_flows = [None]*3, [None]*3, [None]*3, [None]*3
    stock_based_compensations, unearned_revenues, total_rpos, current_rpos = [None]*3, [None]*3, [None]*3, [None]*3
    non_current_rpos, revenue_americas, revenue_europe, revenue_apac = [None]*3, [None]*3, [None]*3, [None]*3

    # --- Manually set CIK ---
    cik_value = 1108524 
    ciks = [cik_value, cik_value, cik_value]

    # --- Step 1: Extract Fiscal Years (from current filing only) ---
    year_match = re.search(r"Fiscal Year Ended January 31,\s+(\d{4})\s+(\d{4})\s+(\d{4})", text_current)
    if year_match:
        years = year_match.groups()
        dates = [datetime.strptime(f"January 31, {y}", "%B %d, %Y").date() for y in years]
        date_ids = [int(d.strftime('%Y%m%d')) for d in dates]
        periods = [f"FY{y}" for y in years]

    # --- Step 2: Extract 3-Year Metrics (from current filing only) ---
    # This entire block of code is unchanged, as you requested.
    revenue_match = re.search(r"Total revenues\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if revenue_match: total_revenues = [float(v.replace(",", "")) for v in revenue_match.groups()]
    cost_revenue_match = re.search(r"Total cost of revenues\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if cost_revenue_match: cost_of_revenues = [float(v.replace(",", "")) for v in cost_revenue_match.groups()]
    op_income_match = re.search(r"Income from operations\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if op_income_match: operating_incomes = [float(v.replace(",", "")) for v in op_income_match.groups()]
    net_income_match = re.search(r"Net income\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if net_income_match: net_incomes = [float(v.replace(",", "")) for v in net_income_match.groups()]
    rd_expense_match = re.search(r"Research and development\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if rd_expense_match: rd_expenses = [float(v.replace(",", "")) for v in rd_expense_match.groups()]
    sm_expense_match = re.search(r"Sales and marketing\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if sm_expense_match: sm_expenses = [float(v.replace(",", "")) for v in sm_expense_match.groups()]
    ga_expense_match = re.search(r"General and administrative\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if ga_expense_match: ga_expenses = [float(v.replace(",", "")) for v in ga_expense_match.groups()]
    sub_rev_match = re.search(r"Revenues:.*?Subscription and support\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if sub_rev_match: subscription_revenues = [float(v.replace(",", "")) for v in sub_rev_match.groups()]
    cost_sub_rev_match = re.search(r"Cost of revenues.*?Subscription and support\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if cost_sub_rev_match: cost_of_subscription_revenues = [float(v.replace(",", "")) for v in cost_sub_rev_match.groups()]
    ps_rev_match = re.search(r"Revenues:.*?Professional services and other\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if ps_rev_match: prof_services_revenues = [float(v.replace(",", "")) for v in ps_rev_match.groups()]
    op_cash_flow_match = re.search(r"Net cash provided by operating activities\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if op_cash_flow_match: operating_cash_flows = [float(v.replace(",", "")) for v in op_cash_flow_match.groups()]
    sbc_match = re.search(r"Stock-based compensation expense\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if sbc_match: stock_based_compensations = [float(v.replace(",", "")) for v in sbc_match.groups()]
    americas_match = re.search(r"Americas\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if americas_match: revenue_americas = [float(v.replace(",", "")) for v in americas_match.groups()]
    europe_match = re.search(r"Europe\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if europe_match: revenue_europe = [float(v.replace(",", "")) for v in europe_match.groups()]
    apac_match = re.search(r"Asia Pacific\s+\$?([\d,]+)\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    if apac_match: revenue_apac = [float(v.replace(",", "")) for v in apac_match.groups()]

    # --- Step 3: MODIFIED LOGIC for 2-Year Metrics ---
    # For these variables, we get FY23 data from the previous filing.

    # Unearned Revenue
    unearned_rev_match_curr = re.search(r"Unearned revenue\s+\$?([\d,]+)\s+\$?([\d,]+)", text_current, re.IGNORECASE)
    unearned_rev_match_prev = re.search(r"Unearned revenue\s+\$?([\d,]+)\s+\$?([\d,]+)", text_previous, re.IGNORECASE)
    if unearned_rev_match_curr and unearned_rev_match_prev:
        unearned_revenues = [
            float(unearned_rev_match_curr.group(1).replace(",", "")),  # FY25 from current
            float(unearned_rev_match_curr.group(2).replace(",", "")),  # FY24 from current
            float(unearned_rev_match_prev.group(2).replace(",", ""))   # FY23 from previous
        ]
    
    # RPO values
    rpo25_match = re.search(r"As of January 31, 2025\s+\$?([\d.]+)\s+\$?([\d.]+)\s+\$?([\d.]+)", text_current)
    rpo24_match = re.search(r"As of January 31, 2024\s+\$?([\d.]+)\s+\$?([\d.]+)\s+\$?([\d.]+)", text_current)
    rpo23_match = re.search(r"As of January 31, 2023\s+\$?([\d.]+)\s+\$?([\d.]+)\s+\$?([\d.]+)", text_previous) # Search previous filing for FY23 RPO
    if rpo25_match and rpo24_match and rpo23_match:
        current_rpos = [float(rpo25_match.group(1))*1000, float(rpo24_match.group(1))*1000, float(rpo23_match.group(1))*1000]
        non_current_rpos = [float(rpo25_match.group(2))*1000, float(rpo24_match.group(2))*1000, float(rpo23_match.group(2))*1000]
        total_rpos = [float(rpo25_match.group(3))*1000, float(rpo24_match.group(3))*1000, float(rpo23_match.group(3))*1000]

    # --- Step 4: Assemble the final dictionary ---
    result = {
        "CIK": ciks, "Period": periods, "Date": dates, "DateID": date_ids,
        "TotalRevenue (millions USD)": total_revenues,
        "SubscriptionRevenue (millions USD)": subscription_revenues,
        "ProfessionalServicesRevenue (millions USD)": prof_services_revenues,
        "Revenue_Americas (millions USD)": revenue_americas,
        "Revenue_Europe (millions USD)": revenue_europe,
        "Revenue_APAC (millions USD)": revenue_apac,
        "CostOfRevenue (millions USD)": cost_of_revenues,
        "CostOfSubscriptionRevenue (millions USD)": cost_of_subscription_revenues,
        "OperatingIncome (millions USD)": operating_incomes,
        "NetIncome (millions USD)": net_incomes,
        "OperatingCashFlow (millions USD)": operating_cash_flows,
        "UnearnedRevenue (millions USD)": unearned_revenues,
        "TotalRPO (millions USD)": total_rpos,
        "CurrentRPO (millions USD)": current_rpos,
        "NonCurrentRPO (millions USD)": non_current_rpos,
        "StockBasedCompensation (millions USD)": stock_based_compensations,
        "RD_Expense (millions USD)": rd_expenses,
        "SM_Expense (millions USD)": sm_expenses,
        "GA_Expense (millions USD)": ga_expenses
    }
    
    return result

In [168]:
data = extract_financial_data(kf1['Item 8'], kf2['Item 8'])


In [169]:
df_y = pd.DataFrame(data).reset_index(drop=True)
df_y

,CIK,Period,Date,DateID,TotalRevenue (millions USD),SubscriptionRevenue (millions USD),ProfessionalServicesRevenue (millions USD),Revenue_Americas (millions USD),Revenue_Europe (millions USD),Revenue_APAC (millions USD),...,NetIncome (millions USD),OperatingCashFlow (millions USD),UnearnedRevenue (millions USD),TotalRPO (millions USD),CurrentRPO (millions USD),NonCurrentRPO (millions USD),StockBasedCompensation (millions USD),RD_Expense (millions USD),SM_Expense (millions USD),GA_Expense (millions USD)
0,1108524,FY2025,2025-01-31,20250131,37895.0,35679.0,2216.0,25143.0,8891.0,3861.0,...,6197.0,13092.0,20743.0,63400.0,30200.0,33200.0,3183.0,5493.0,13257.0,2836.0
1,1108524,FY2024,2024-01-31,20240131,34857.0,32537.0,2320.0,23289.0,8128.0,3440.0,...,4136.0,10234.0,19003.0,56900.0,27600.0,29300.0,2787.0,4906.0,12877.0,2534.0
2,1108524,FY2023,2023-01-31,20230131,31352.0,29021.0,2331.0,21250.0,7163.0,2939.0,...,208.0,7111.0,17376.0,48600.0,24600.0,24000.0,3279.0,5055.0,13526.0,2553.0


In [140]:
import re
from datetime import datetime

def extract_financial_data_10q(filing_text: str):
    """
    Extracts key financial data from the text of a 10-Q filing for the
    single most recent quarterly period presented.

    Args:
        filing_text (str): The raw text of the financial statements from a 10-Q filing.

    Returns:
        dict: A dictionary where keys are the variable names and values are lists
              containing a single value for the most recent quarter.
    """
    # 1. Normalize whitespace to make parsing consistent
    text = re.sub(r'\s+', ' ', filing_text)

    # --- Initialize lists to store all extracted data ---
    ciks, periods, dates, date_ids = [], [], [], []
    total_revenues, cost_of_revenues, operating_incomes, net_incomes = [], [], [], []
    rd_expenses, sm_expenses, ga_expenses = [], [], []
    subscription_revenues, cost_of_subscription_revenues = [], []
    prof_services_revenues, operating_cash_flows = [], []
    stock_based_compensations, unearned_revenues = [], []
    total_rpos, current_rpos, non_current_rpos = [], [], []
    revenue_americas, revenue_europe, revenue_apac = [], [], []
    
    report_date = None # Initialize report_date

    # --- Step 1: Manually set CIK and extract Date to derive Period ---
    ciks.append(1108524)
    
    date_header_match = re.search(
        r"Three Months Ended\s+([A-Za-z]+\s+\d{1,2}),.*?(\d{4})",
        text,
        re.IGNORECASE
    )

    if date_header_match:
        month_day, year_str = date_header_match.groups()
        report_date = datetime.strptime(f"{month_day}, {year_str}", "%B %d, %Y").date()
        
        dates.append(report_date)
        date_ids.append(int(report_date.strftime('%Y%m%d')))

        # Calculate the fiscal period (Salesforce FY ends Jan 31)
        month = report_date.month
        fiscal_year = report_date.year + 1 if month > 1 else report_date.year
        
        if month in [2, 3, 4]: quarter = "Q1"
        elif month in [5, 6, 7]: quarter = "Q2"
        elif month in [8, 9, 10]: quarter = "Q3"
        else: quarter = "Q4"
            
        periods.append(f"{quarter} FY{fiscal_year}")

    # --- Step 2: Extract financial metrics for the most recent quarter ---
    
    # Revenue items
    revenue_match = re.search(r"Total revenues\s+\$?([\d,]+)", text, re.IGNORECASE)
    if revenue_match: total_revenues.append(float(revenue_match.group(1).replace(",", "")))
        
    sub_rev_match = re.search(r"Revenues:.*?Subscription and support\s+\$?([\d,]+)", text, re.IGNORECASE)
    if sub_rev_match: subscription_revenues.append(float(sub_rev_match.group(1).replace(",", "")))
        
    ps_rev_match = re.search(r"Professional services and other\s+\$?([\d,]+)", text, re.IGNORECASE)
    if ps_rev_match: prof_services_revenues.append(float(ps_rev_match.group(1).replace(",", "")))

    # Revenue by Geography
    americas_match = re.search(r"Americas\s+\$?([\d,]+)", text, re.IGNORECASE)
    if americas_match: revenue_americas.append(float(americas_match.group(1).replace(",", "")))

    europe_match = re.search(r"Europe\s+\$?([\d,]+)", text, re.IGNORECASE)
    if europe_match: revenue_europe.append(float(europe_match.group(1).replace(",", "")))

    apac_match = re.search(r"Asia Pacific\s+\$?([\d,]+)", text, re.IGNORECASE)
    if apac_match: revenue_apac.append(float(apac_match.group(1).replace(",", "")))

    # Cost of Revenue items
    cost_revenue_match = re.search(r"Total cost of revenues\s+\$?([\d,]+)", text, re.IGNORECASE)
    if cost_revenue_match: cost_of_revenues.append(float(cost_revenue_match.group(1).replace(",", "")))
        
    cost_sub_rev_match = re.search(r"Cost of revenues.*?Subscription and support\s+\$?([\d,]+)", text, re.IGNORECASE)
    if cost_sub_rev_match: cost_of_subscription_revenues.append(float(cost_sub_rev_match.group(1).replace(",", "")))
        
    # Operating Expenses and Income items
    op_income_match = re.search(r"Income from operations\s+\$?([\d,]+)", text, re.IGNORECASE)
    if op_income_match: operating_incomes.append(float(op_income_match.group(1).replace(",", "")))
        
    net_income_match = re.search(r"Net income\s+\$?([\d,]+)", text, re.IGNORECASE)
    if net_income_match: net_incomes.append(float(net_income_match.group(1).replace(",", "")))
        
    rd_expense_match = re.search(r"Research and development\s+\$?([\d,]+)", text, re.IGNORECASE)
    if rd_expense_match: rd_expenses.append(float(rd_expense_match.group(1).replace(",", "")))

    sm_expense_match = re.search(r"Sales and marketing\s+\$?([\d,]+)", text, re.IGNORECASE)
    if not sm_expense_match: sm_expense_match = re.search(r"Marketing and sales\s+\$?([\d,]+)", text, re.IGNORECASE)
    if sm_expense_match: sm_expenses.append(float(sm_expense_match.group(1).replace(",", "")))

    ga_expense_match = re.search(r"General and administrative\s+\$?([\d,]+)", text, re.IGNORECASE)
    if ga_expense_match: ga_expenses.append(float(ga_expense_match.group(1).replace(",", "")))
        
    # Cash Flow and Balance Sheet items
    op_cash_flow_match = re.search(r"Net cash provided by operating activities\s+\$?([\d,]+)", text, re.IGNORECASE)
    if op_cash_flow_match: operating_cash_flows.append(float(op_cash_flow_match.group(1).replace(",", "")))
        
    # MODIFIED: Anchored search for Stock-Based Compensation
    sbc_match = re.search(r"Operating activities:.*?Stock-based compensation expense\s+\$?([\d,]+)", text, re.IGNORECASE)
    if sbc_match: stock_based_compensations.append(float(sbc_match.group(1).replace(",", "")))

    unearned_rev_match = re.search(r"Unearned revenue\s+\$?([\d,]+)", text, re.IGNORECASE)
    if unearned_rev_match: unearned_revenues.append(float(unearned_rev_match.group(1).replace(",", "")))

    # RPO items (with unit conversion)
    if report_date:
        date_str_for_rpo = report_date.strftime("%B %d, %Y")
        rpo_pattern = rf"As of {re.escape(date_str_for_rpo)}\s+\$?([\d.]+)\s+\$?([\d.]+)\s+\$?([\d.]+)"
        rpo_match = re.search(rpo_pattern, text)
        
        if rpo_match:
            current_rpo_val = float(rpo_match.group(1)) * 1000
            non_current_rpo_val = float(rpo_match.group(2)) * 1000
            total_rpo_val = float(rpo_match.group(3)) * 1000
            current_rpos.append(current_rpo_val)
            non_current_rpos.append(non_current_rpo_val)
            total_rpos.append(total_rpo_val)

    # --- Step 3: Assemble the final dictionary with units ---
    result = {
        "CIK": ciks, "Period": periods, "Date": dates, "DateID": date_ids,
        "TotalRevenue (millions USD)": total_revenues,
        "SubscriptionRevenue (millions USD)": subscription_revenues,
        "ProfessionalServicesRevenue (millions USD)": prof_services_revenues,
        "Revenue_Americas (millions USD)": revenue_americas,
        "Revenue_Europe (millions USD)": revenue_europe,
        "Revenue_APAC (millions USD)": revenue_apac,
        "CostOfRevenue (millions USD)": cost_of_revenues,
        "CostOfSubscriptionRevenue (millions USD)": cost_of_subscription_revenues,
        "OperatingIncome (millions USD)": operating_incomes,
        "NetIncome (millions USD)": net_incomes,
        "OperatingCashFlow (millions USD)": operating_cash_flows,
        "UnearnedRevenue (millions USD)": unearned_revenues,
        "TotalRPO (millions USD)": total_rpos,
        "CurrentRPO (millions USD)": current_rpos,
        "NonCurrentRPO (millions USD)": non_current_rpos,
        "StockBasedCompensation (millions USD)": stock_based_compensations,
        "RD_Expense (millions USD)": rd_expenses,
        "SM_Expense (millions USD)": sm_expenses,
        "GA_Expense (millions USD)": ga_expenses,
    }
    return result


In [141]:
data_q1 = extract_financial_data_10q(qf1['Item 1'])
data_q2 = extract_financial_data_10q(qf2['Item 1'])
data_q3 = extract_financial_data_10q(qf3['Item 1'])
data_q4 = extract_financial_data_10q(qf4['Item 1'])
data_q5 = extract_financial_data_10q(qf5['Item 1'])
data_q6 = extract_financial_data_10q(qf6['Item 1'])
data_q7 = extract_financial_data_10q(qf7['Item 1'])
data_q8 = extract_financial_data_10q(qf8['Item 1'])

In [170]:
all_data = [
    data_q1,
    data_q2,
    data_q3,
    data_q4,
    data_q5,
    data_q6,
    data_q7,
    data_q8,
]

list_of_dfs = [pd.DataFrame(data) for data in all_data]

final_df = pd.concat(list_of_dfs, ignore_index=True)
final_df = final_df.sort_values(by='Date', ascending=True).reset_index(drop=True)

In [171]:
final_df

,CIK,Period,Date,DateID,TotalRevenue (millions USD),SubscriptionRevenue (millions USD),ProfessionalServicesRevenue (millions USD),Revenue_Americas (millions USD),Revenue_Europe (millions USD),Revenue_APAC (millions USD),...,NetIncome (millions USD),OperatingCashFlow (millions USD),UnearnedRevenue (millions USD),TotalRPO (millions USD),CurrentRPO (millions USD),NonCurrentRPO (millions USD),StockBasedCompensation (millions USD),RD_Expense (millions USD),SM_Expense (millions USD),GA_Expense (millions USD)
0,1108524,Q1 FY2024,2023-04-30,20230430,8247.0,7642.0,605.0,5482.0,1951.0,814.0,...,199.0,4491.0,15121.0,46700.0,24100.0,22600.0,696.0,1207.0,3154.0,638.0
1,1108524,Q2 FY2024,2023-07-31,20230731,8603.0,8006.0,597.0,5769.0,1974.0,860.0,...,1267.0,808.0,14237.0,46600.0,24100.0,22500.0,724.0,1220.0,3113.0,632.0
2,1108524,Q3 FY2024,2023-10-31,20231031,8720.0,8141.0,579.0,5862.0,1998.0,860.0,...,1224.0,1532.0,12564.0,48300.0,23900.0,24400.0,693.0,1204.0,3173.0,632.0
3,1108524,Q1 FY2025,2024-04-30,20240430,9133.0,8585.0,548.0,6062.0,2145.0,926.0,...,1533.0,6247.0,16061.0,53900.0,26400.0,27500.0,750.0,1368.0,3239.0,647.0
4,1108524,Q2 FY2025,2024-07-31,20240731,9325.0,8764.0,561.0,6201.0,2184.0,940.0,...,1429.0,892.0,15222.0,53500.0,26500.0,27000.0,810.0,1349.0,3224.0,711.0
5,1108524,Q3 FY2025,2024-10-31,20241031,9444.0,8879.0,565.0,6220.0,2228.0,996.0,...,1527.0,1983.0,13472.0,53100.0,26400.0,26700.0,820.0,1356.0,3323.0,711.0
6,1108524,Q1 FY2026,2025-04-30,20250430,9829.0,9297.0,532.0,6469.0,2337.0,1023.0,...,1541.0,6476.0,17799.0,60900.0,29600.0,31300.0,814.0,1460.0,3429.0,697.0
7,1108524,Q2 FY2026,2025-07-31,20250731,10236.0,9690.0,546.0,6736.0,2429.0,1071.0,...,1887.0,740.0,16555.0,59900.0,29400.0,30500.0,793.0,1481.0,3443.0,734.0


In [139]:
print(qf8['Item 1'])

ITEM 1. FINANCIAL STATEMENTS
Salesforce, Inc.
Condensed Consolidated Balance Sheets
(in millions)

                                                                         April 30, 2023                 January 31, 2023
Assets                                                                      (unaudited)                                 
Current assets:                                                                                                         
Cash and cash equivalents                                                 $9,155                          $7,016        
Marketable securities                                                      4,822                           5,492        
Accounts receivable, net                                                   4,632                          10,755        
Costs capitalized to obtain revenue contracts, net                         1,772                           1,776        
Prepaid expenses and other current assets             

In [172]:
# 1. Define "Flow" and "Stock" columns
# Flow columns are cumulative, so Q4 is calculated.
flow_cols = [
    'TotalRevenue (millions USD)', 'SubscriptionRevenue (millions USD)',
    'ProfessionalServicesRevenue (millions USD)', 'Revenue_Americas (millions USD)',
    'Revenue_Europe (millions USD)', 'Revenue_APAC (millions USD)',
    'CostOfRevenue (millions USD)', 'CostOfSubscriptionRevenue (millions USD)',
    'OperatingIncome (millions USD)', 'NetIncome (millions USD)',
    'OperatingCashFlow (millions USD)', 'StockBasedCompensation (millions USD)',
    'RD_Expense (millions USD)', 'SM_Expense (millions USD)', 'GA_Expense (millions USD)'
]

# Stock columns are point-in-time, so Q4 value is the year-end value.
stock_cols = [
    'UnearnedRevenue (millions USD)', 'TotalRPO (millions USD)',
    'CurrentRPO (millions USD)', 'NonCurrentRPO (millions USD)'
]

# List of fiscal years for which we need to calculate Q4
fiscal_years_to_calc = [2024, 2025]
new_q4_rows = []

# 2. Loop through each fiscal year to calculate its Q4
for year in fiscal_years_to_calc:    
    # Get the full fiscal year data
    annual_data = df_y[df_y['Period'] == f"FY{year}"].iloc[0]
    
    # Get the first three quarters for that fiscal year
    q1_q3_data = final_df[final_df['Period'].str.contains(f"FY{year}")]
    
    # --- Start building the new Q4 row ---
    q4_row = {}
    
    # Calculate all "Flow" metrics
    for col in flow_cols:
        sum_of_quarters = q1_q3_data[col].sum()
        q4_value = annual_data[col] - sum_of_quarters
        q4_row[col] = q4_value
        
    # Directly assign all "Stock" metrics from the year-end data
    for col in stock_cols:
        q4_row[col] = annual_data[col]
        
    # Add identifier columns
    q4_row['CIK'] = annual_data['CIK']
    q4_row['Period'] = f"Q4 FY{year}"
    q4_row['Date'] = annual_data['Date']
    q4_row['DateID'] = annual_data['DateID']
    
    new_q4_rows.append(q4_row)


df_q4 = pd.DataFrame(new_q4_rows)
df_full_quarterly = pd.concat([final_df, df_q4], ignore_index=True)

# Sort the final DataFrame chronologically and reset the index
df_full_quarterly = df_full_quarterly.sort_values(by='Date').reset_index(drop=True)

In [ ]:
df_full_quarterly

,CIK,Period,Date,DateID,TotalRevenue (millions USD),SubscriptionRevenue (millions USD),ProfessionalServicesRevenue (millions USD),Revenue_Americas (millions USD),Revenue_Europe (millions USD),Revenue_APAC (millions USD),...,NetIncome (millions USD),OperatingCashFlow (millions USD),UnearnedRevenue (millions USD),TotalRPO (millions USD),CurrentRPO (millions USD),NonCurrentRPO (millions USD),StockBasedCompensation (millions USD),RD_Expense (millions USD),SM_Expense (millions USD),GA_Expense (millions USD)
0,1108524,Q1 FY2024,2023-04-30,20230430,8247.0,7642.0,605.0,5482.0,1951.0,814.0,...,199.0,4491.0,15121.0,46700.0,24100.0,22600.0,696.0,1207.0,3154.0,638.0
1,1108524,Q2 FY2024,2023-07-31,20230731,8603.0,8006.0,597.0,5769.0,1974.0,860.0,...,1267.0,808.0,14237.0,46600.0,24100.0,22500.0,724.0,1220.0,3113.0,632.0
2,1108524,Q3 FY2024,2023-10-31,20231031,8720.0,8141.0,579.0,5862.0,1998.0,860.0,...,1224.0,1532.0,12564.0,48300.0,23900.0,24400.0,693.0,1204.0,3173.0,632.0
3,1108524,Q4 FY2024,2024-01-31,20240131,9287.0,8748.0,539.0,6176.0,2205.0,906.0,...,1446.0,3403.0,19003.0,56900.0,27600.0,29300.0,674.0,1275.0,3437.0,632.0
4,1108524,Q1 FY2025,2024-04-30,20240430,9133.0,8585.0,548.0,6062.0,2145.0,926.0,...,1533.0,6247.0,16061.0,53900.0,26400.0,27500.0,750.0,1368.0,3239.0,647.0
5,1108524,Q2 FY2025,2024-07-31,20240731,9325.0,8764.0,561.0,6201.0,2184.0,940.0,...,1429.0,892.0,15222.0,53500.0,26500.0,27000.0,810.0,1349.0,3224.0,711.0
6,1108524,Q3 FY2025,2024-10-31,20241031,9444.0,8879.0,565.0,6220.0,2228.0,996.0,...,1527.0,1983.0,13472.0,53100.0,26400.0,26700.0,820.0,1356.0,3323.0,711.0
7,1108524,Q4 FY2025,2025-01-31,20250131,9993.0,9451.0,542.0,6660.0,2334.0,999.0,...,1708.0,3970.0,20743.0,63400.0,30200.0,33200.0,803.0,1420.0,3471.0,767.0
8,1108524,Q1 FY2026,2025-04-30,20250430,9829.0,9297.0,532.0,6469.0,2337.0,1023.0,...,1541.0,6476.0,17799.0,60900.0,29600.0,31300.0,814.0,1460.0,3429.0,697.0
9,1108524,Q2 FY2026,2025-07-31,20250731,10236.0,9690.0,546.0,6736.0,2429.0,1071.0,...,1887.0,740.0,16555.0,59900.0,29400.0,30500.0,793.0,1481.0,3443.0,734.0


: 